In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
import hyperopt
from sklearn.model_selection import train_test_split,cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer
import colorama

## 载入数据

In [2]:
X_train = pd.read_csv('C:/Users/Boniface/Desktop/房租预测/datasets/cluster_train.csv')
Y_train = X_train.pop('tradeMoney')
X_train.pop('Unnamed: 0')
Train = xgb.DMatrix(X_train, label = Y_train)

D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
D:\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [20]:
obj_call_count = 0
cur_best_score = 0
log_writer = open( 'XGB-hyperopt-log.txt', 'w' )
def objective(space):
    global obj_call_count, cur_best_score
    obj_call_count += 1
    
    xgb_params = {
        'eval_metric':'rmse',
        'objective' : 'reg:gamma',
        'booster':'gbtree',
        'max_depth': int(space['max_depth']),
        'gamma': space['gamma'],
        'colsample_bytree': space['colsample_bytree'],
        'colsample_bylevel' : space['colsample_bylevel'],
        'eta':space['eta'],
        'min_child_weight' : space['min_child_weight'],
        'subsample' : space['subsample'],
        'learning_rate' : space['learning_rate'],
        'min_child_weight':space['min_child_weight']
    }
    
    print('\nXGB objective call #{} cur_best_score={:7.5f}'.format(obj_call_count,cur_best_score) )
    
    kf = KFold(n_splits=3, shuffle=True, random_state=0)
    out_of_fold = np.zeros(len(X_train))
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        Train = xgb.DMatrix(X_train.iloc[train_idx],Y_train[train_idx])
        Val =   xgb.DMatrix(X_train.iloc[val_idx], label=Y_train[val_idx])
        
        evallist = [(Train, 'train'),(Val, 'valid')]
        xgb_modle = xgb.train(xgb_params, Train, num_boost_round=1, evals = evallist)
        
        out_of_fold[val_idx] = xgb_modle.predict(xgb.DMatrix(X_train.iloc[val_idx]))
        score = r2_score(out_of_fold, Y_train)
        print('val_r2_score={}'.format(score))
        
    log_writer.write('score={}\n'.format(score))
    log_writer.flush()

    if score>cur_best_score:
        cur_best_score = score
        print(colorama.Fore.GREEN + 'NEW BEST SCORE={}'.format(cur_best_score) + colorama.Fore.RESET)
        
    return {'loss': -score, 'status': STATUS_OK}

space = {
    'learning_rate' : hp.uniform('learning_rate',0,0.4),
    'gamma': hp.uniform('gamma', 0, 0.5),
    'min_child_weight':hp.uniform('min_child_weight',0,2),
    'max_depth': hp.uniform('max_depth',4,6),
    'lambda': hp.uniform('lambda',15,20),
    'subsample':hp.uniform('subsample',0,0.4),
    'colsample_bytree':hp.uniform('colsample_bytree',0,0.3),
    'colsample_bylevel':hp.uniform('colsample_bylevel',0,0.9),
    'eta': hp.uniform('eta',0,2)
}

trials = Trials()
best = hyperopt.fmin(fn=objective,
                     space=space,
                     algo=tpe.suggest,
                     max_evals=10,
                     trials=trials,
                     verbose=1)

print('-'*50)
print('The best params:')
print( best )
print('\n\n')
log_writer.write('best_params={}\n'.format(best))
log_writer.flush()

                                                                                                            
XGB objective call #1 cur_best_score=0.00000
  0%|                                                                | 0/10 [00:00<?, ?trial/s, best loss=?]

D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[0]	train-rmse:5063.78	valid-rmse:5012.07                                                                   

val_r2_score=-390889899.16350937                                                                            
  0%|                                                                | 0/10 [00:01<?, ?trial/s, best loss=?]

D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[0]	train-rmse:5049.25	valid-rmse:5041.3                                                                    

val_r2_score=-390880081.4031657                                                                             
  0%|                                                                | 0/10 [00:01<?, ?trial/s, best loss=?]

...

[0]	train-rmse:5063.78	valid-rmse:5012.06                                                                   

val_r2_score=-378487571.16812176                                                                            
 90%|██████████████████████████████▌   | 9/10 [00:23<00:02,  2.55s/trial, best loss: 1.6787362591218747e+18]

D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[0]	train-rmse:5049.25	valid-rmse:5041.29                                                                   

val_r2_score=-378477766.54979473                                                                            
 90%|██████████████████████████████▌   | 9/10 [00:24<00:02,  2.55s/trial, best loss: 1.6787362591218747e+18]

D:\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \



[0]	train-rmse:5026.7	valid-rmse:5086.16                                                                    

val_r2_score=-3.2258502548969285e+22                                                                        
100%|█████████████████████████████████| 10/10 [00:25<00:00,  2.55s/trial, best loss: 1.6787362591218747e+18]
--------------------------------------------------
The best params:
{'colsample_bylevel': 0.48988374231135856, 'colsample_bytree': 0.2130437213015567, 'eta': 1.9134853497647035, 'gamma': 0.07393498485374711, 'lambda': 18.223566813917763, 'learning_rate': 0.34434173612498836, 'max_depth': 4.8943516410959305, 'min_child_weight': 0.3633108415279549, 'subsample': 0.25235555808827564}





In [6]:
test = pd.read_csv('C:/Users/Boniface/Desktop/房租预测/datasets/cluster_test.csv')
test.pop('Unnamed: 0')
test.pop('tradeMoney')

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
2464   NaN
2465   NaN
2466   NaN
2467   NaN
2468   NaN
Name: tradeMoney, Length: 2469, dtype: float64

In [4]:
xgb_params = {'colsample_bylevel': 0.7561534982217195, 'colsample_bytree': 0.29553715278515313, 'eta': 1.2527570325666055, 
              'gamma': 0.13580374578821358, 'lambda': 15.547715254742997, 'learning_rate': 0.049033226916099903, 
              'max_depth': int(5.34805217981379), 'min_child_weight': 1.4908292047443312, 'subsample': 0.39948434547036404,
         'eval_metric':'rmse',
        'objective' : 'reg:gamma',
        'booster':'gbtree','tree_method' : 'gpu_hist'}

xgb_modle = xgb.train(xgb_params, Train, num_boost_round=1200)

In [7]:
tradeMoney_predict = xgb_modle.predict(xgb.DMatrix(test))

array([4256.559 , 5209.062 , 5370.614 , ..., 5223.2095, 5407.6797,
       3769.1033], dtype=float32)

In [8]:
judge = pd.read_csv('C:/Users/Boniface/Desktop/房租预测/datasets/评分文件/sub_a_913.csv')
score = r2_score(judge, tradeMoney_predict)
print(score)

0.9739413074546406


In [10]:
tradeMoney_predict = pd.DataFrame(tradeMoney_predict)

In [11]:
tradeMoney_predict.to_csv('xgb_tradeMoney_predict.csv')

In [19]:
params={'booster':'gbtree',
	    'eval_metric':'auc',
	    'gamma':0.1,
	    'min_child_weight':1.1,
	    'max_depth':5,
	    'lambda':10,
	    'subsample':0.7,
	    'colsample_bytree':0.7,
	    'colsample_bylevel':0.7,
	    'eta': 0.01,
	    'tree_method':'exact',
	    'seed':0,
	    'nthread':12
	    }

In [25]:
model_xgb = xgb.train(params, Train, num_boost_round=1000)

In [36]:
test = pd.read_csv('C:/Users/Boniface/Desktop/房租预测/datasets/cluster_test.csv')
test.pop('Unnamed: 0')
test.pop('tradeMoney')
Test = xgb.DMatrix(test)

In [37]:
tradeMoney_predict = model_xgb.predict(Test)

In [38]:
tradeMoney_predict

array([4240.1235, 5151.479 , 6127.673 , ..., 4987.579 , 5956.1445,
       3912.988 ], dtype=float32)

## 调参

In [59]:
N_HYPEROPT_PROBES = 10
HYPEROPT_ALGO = tpe.suggest 

def get_xgb_params(space):
    xgb_params = dict()
    xgb_params['booster'] = 'gbtree'
    xgb_params['eval_metric'] = 'rmse'
    xgb_params['seed'] = 0
    xgb_params['gamma'] = space['gamma']
    xgb_params['min_child_weight'] = space['min_child_weight']
    xgb_params['max_depth'] = int(space['max_depth'])
    xgb_params['lambda'] = space['lambda']
    xgb_params['subsample'] = space['subsample']
    xgb_params['colsample_bytree'] = space['colsample_bylevel']
    return xgb_params

space = {
    'gamma': hp.uniform('gamma', 0, 0.5),
    'min_child_weight':hp.quniform('min_child_weight',0, 2, 1),
    'max_depth': hp.uniform('max_depth',0,10),
    'lambda': hp.uniform('lambda',0,20),
    'subsample':hp.uniform('subsample',0,1),
    'colsample_bytree':hp.uniform('colsample_bytree',0,1),
    'colsample_bylevel':hp.uniform('colsample_bylevel',0,1),
    'eta': hp.uniform('eta',0,1)
}

X_train = pd.read_csv('C:/Users/Boniface/Desktop/房租预测/datasets/cluster_train.csv')
Y_train = X_train.pop('tradeMoney')
X_train.pop('Unnamed: 0')

def RUN(space):
    xgb_params = get_xgb_params(space)
    Train = xgb.DMatrix(X_train, label=Y_train)
    model_xgb = xgb.train(xgb_params, Train, num_boost_round=1000)
    return 'OK'
    
trials = Trials()
best = hyperopt.fmin(fn=RUN,
                     space=space,
                     algo=HYPEROPT_ALGO,
                     max_evals=N_HYPEROPT_PROBES,
                     trials=trials,
                     verbose=1)
print('-'*50)
print('The best params:')
print( best )
print('\n\n')

  0%|                                                                           | 0/10 [00:00<?, ?trial/s, best loss=?]

job exception: dictionary update sequence element #0 has length 1; 2 is required



  0%|                                                                           | 0/10 [00:44<?, ?trial/s, best loss=?]


ValueError: dictionary update sequence element #0 has length 1; 2 is required